In [ ]:
from fastai.vision.all import *
import numpy as np
import sys
import os
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
import fastai
print(fastai.__version__)

In [ ]:
sys.path.append("../input/effnet-pytorch/EfficientNet-PyTorch-master")

In [ ]:
#!cp -rf ../input/effnet-pytorch/EfficientNet-PyTorch-master /kaggle/working/

In [ ]:

#os.chdir("/kaggle/working/EfficientNet-PyTorch-master")

In [ ]:
 
#!pip install -e .

In [ ]:
os.chdir("/kaggle/working")

In [ ]:
'''
m = resnet34()
m = nn.Sequential(*list(m.children())[:-2])
tst = DynamicUnet(m, 2, (128,128), norm_type=None)
x = cast(torch.randn(2, 3, 128, 128), TensorImage)
y = tst(x)
'''

In [ ]:
#print( y.shape)

In [ ]:
import pandas as pd
import os
import random
df = pd.read_csv('/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv', header=0, names=['id','value'], dtype=object)
df = df[~df.id.isin(["00109", "00123", "00709"])]

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
#https://stackoverflow.com/a/4836734/8245487
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

In [ ]:
import numpy as np
np.seterr(divide='ignore',invalid='ignore')

In [ ]:
import os
import pydicom
import pandas as pd
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm import tqdm
import binascii
from PIL import Image

INPUT = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification'

if not os.path.exists('./train'):
    os.makedirs('./train')
    

if not os.path.exists('./test'):
    os.makedirs('./test')


def get_dicom_files(input_dir, dataset='train'):
    for subdir, dirs, files in os.walk(f"{input_dir}/{dataset}"):
        if len(files) == 0:
            continue
        filename = natural_sort(files)[len(files)//2] #take middle most image -- FLAIR DCM file per training item.
        filepath = os.path.join(subdir, filename)
        
        if filepath.endswith(".dcm") and "FLAIR" in filepath:
            cur_id = subdir.split('/')[-2]
            outpath = os.path.join(f'./{dataset}',f'{cur_id}.png')
            
            process_dicom(filepath, outpath)

def process_dicom(path, outpath):
    dicom = pydicom.read_file(path)
    data = apply_voi_lut(dicom.pixel_array, dicom)
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
    height = len(data)
    width = len(data[0])
    
    pixels_out = []
    for row in data:
        pixels_out.extend(row)
    assert(len(pixels_out) == height * width)
    
    image_out = Image.new('L', (width, height))
    image_out.putdata(pixels_out)
    image_out.save(outpath)

get_dicom_files(INPUT, 'train')
get_dicom_files(INPUT, 'test')


#     final = pd.DataFrame(final)
#     final.to_csv(f"{args['output']}/dicom_meta_{args['dataset']}.csv", index=False)

In [ ]:
for id_num in df.id:
    full_path = './train/{}.png'.format(id_num)
    df.loc[df.id == id_num, 'file'] = full_path
    

In [ ]:
df

In [ ]:
#dls = ImageDataLoaders.from_df(df, item_tfms=Resize(224), bs=64, label_col =1, fn_col=2, path='')

In [ ]:
#dls = ImageDataLoaders.from_df(df, item_tfms=Resize(224), bs=16, label_col =1, fn_col=2, path='')

In [ ]:
dls = ImageDataLoaders.from_df(df, item_tfms=Resize(224), bs=8, label_col =1, fn_col=2, path='')

In [ ]:
dls.show_batch()

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, pretrained=False):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

In [ ]:
class Net2(nn.Module):
    def __init__(self, pretrained=False):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import seaborn as sns

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.layers import Input, Activation, BatchNormalization, LeakyReLU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import losses, optimizers

In [ ]:

#learn = cnn_learner(dls, Net, metrics=[error_rate, accuracy], model_dir="/tmp/model/").to_fp16()

In [ ]:
import sys
sys.path.append("../input/pretrained-model-weights-pytorch")

In [ ]:
#!mkdir -p /tmp/model

In [ ]:
!mkdir -p /root/.cache/torch/hub/checkpoints/

In [ ]:
#!mkdir -p /tmp/model

In [ ]:
!cp  /kaggle/input/pytorch-pretrained-models/vgg16_bn-6c64b313.pth  /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth

In [ ]:
!cp  /kaggle/input/resnet50-pytorch-pretrained/resnet50-19c8e357.pth  /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth

In [ ]:
!cp  /kaggle/input/pytorch-pretrained-models/resnet101-5d3b4d8f.pth  /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth

In [ ]:
!cp  /kaggle//input/pytorch-pretrained-models/vgg19_bn-c79401a0.pth  /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth

In [ ]:
!cp  /kaggle/input/densenet121/densenet121-a639ec97.pth  /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth

In [ ]:
!cp  /kaggle/input/pytorch-pretrained-models/resnet152-b121ed2d.pth  /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth

In [ ]:
#learn = cnn_learner(dls, vgg16_bn, metrics=[error_rate, accuracy]).to_fp16()

In [ ]:
#learn = cnn_learner(dls, resnet101, metrics=[error_rate, accuracy]).to_fp16()

In [ ]:
#learn =cnn_learner(dls,xresnet50 , metrics=[error_rate, accuracy] ).to_fp16()

In [ ]:
#learn = cnn_learner(dls, densenet121, metrics=[error_rate, accuracy]).to_fp16()

In [ ]:
''' 
from efficientnet_pytorch import EfficientNet
model_name = 'efficientnet-b2'
def getModel():
    #model = EfficientNet.from_pretrained(model_name)
    model = EfficientNet.from_name(model_name)
    #model._bn1 = nn.Identity()
    #model._fc = nn.Linear(1408,data.c)
    model._fc = nn.Linear(1408,2)
    return model
model=getModel()

!cp '/kaggle/input/effnetb2-4d6/efficientnet-b2-8bb594d6.pth' '/root/.cache/torch/hub/checkpoints/efficientnet-b2-8bb594d6.pth'
 '''

In [ ]:
!cp '/kaggle/input/fastai-xresnet50/xrn50_940.pth' '/root/.cache/torch/hub/checkpoints/xrn50_940.pth'
 
 

In [ ]:
learn =cnn_learner(dls,xresnet152 , metrics=[error_rate, accuracy] ).to_fp16()

In [ ]:
learn.lr_find()

In [ ]:
#learn.fit_one_cycle(100, lr_max=1e-2, cbs=ShortEpochCallback())

In [ ]:
learn.fit_one_cycle(20, lr_max=1e-2)

In [ ]:
learn.fit(10)

In [ ]:
learn.show_results()

In [ ]:
df_test = pd.DataFrame(columns=['id', 'value'])
df_test.id = os.listdir("/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/test/")

In [ ]:
for id_num in df_test.id:
    full_path = './test/{}.png'.format(id_num)
    prediction = learn.predict(full_path)
    print(prediction)
    probability = prediction[2][1].item()
    print(probability)
    df_test.loc[df_test.id==id_num, 'value'] = probability

In [ ]:
df_test.head()

In [ ]:
df_test.value.min()

In [ ]:
df_test.value.max()

In [ ]:
df_test.rename(columns={'id':'BraTS21ID','value':'MGMT_value'}).to_csv('submission.csv', index=False)